In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


SEED = 314159
TRAIN_TEST_SPLIT = 0.80

data_path = r"C:\Users\nikol_ri8fhbe\Documents\ml"

In [ ]:
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, 
                           n_redundant=5, random_state=42)

In [ ]:
plt.scatter(X[:, 2], X[:, 3], c=y)

# Ансамбли: бустинги


Бустинг строится последовательно: каждое следующее дерево в нем обучается на основе результатов предыдущего, пытаясь уменьшить его ошибку. Как следствие, композиция будет иметь меньшее смещение, чем исходные базовые модели. Поэтому логично в качестве базовых моделей использовать те, которые изначально обладают небольшим разбросом и высоким смещением. Вопрос: какие это будут модели?
Еще одно соображение для выбора сильно смещенных моделей в том, что они банально быстрее учатся. Так как невозможно распараллелить обучение базовых моделей, то скорость их настройки становится серьезным вопросом. 

Что интересно, бустинги не очень хорошо работают с однородными данными - поэтому их нечасто применяют для текстов.

Расссмотрим квадратичную функцию потерь и композицию следующего вида: $ a = b_1 +  b_2 + ... + b_N $
Обучим только одно дерево $ a = b_1 $. Найдем примеры, для которых оно ошибается в  предсказании. Обучим для них еще одно дерево - $ b_2 $, которое будет предсказывать ошибку первого. Будем повторять это, пока не наберем K деревьев. Примерно так на верхнем уровне обучается бустинг. 


In [ ]:
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

def eval_classifier(clf):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=43)
    n_scores = cross_val_score(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return np.mean(n_scores), np.std(n_scores)

In [ ]:
results_accuracy = pd.DataFrame(0.0,
                                columns=["W/o ensembling", 'Bagging', "Bagging_with_mf", 'AdaBoost'],
                                index=['deep DTC', '1-level DTC', 'LR', 'SVC'])

In [ ]:
acc_mean, acc_std = eval_classifier(DecisionTreeClassifier())

results_accuracy.loc['deep DTC', 'W/o ensembling'] = acc_mean
print(f"{acc_mean:.2f}, +- {acc_std:.2f}")

In [ ]:
acc_mean, acc_std = eval_classifier(DecisionTreeClassifier(max_depth=1))

results_accuracy.loc['1-level DTC', 'W/o ensembling'] = acc_mean
print(f"{acc_mean:.2f} +- {acc_std:.2f}")

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
acc_mean, acc_std = eval_classifier(
    BaggingClassifier(DecisionTreeClassifier(), n_estimators=10, 
                      max_samples=1.0, max_features=1.0, 
                      bootstrap=True, bootstrap_features=False))

results_accuracy.loc['deep DTC', 'Bagging'] = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

In [ ]:
acc_mean, acc_std = eval_classifier(
    BaggingClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=10, 
                      max_samples=1.0, max_features=1.0, 
                      bootstrap=True, bootstrap_features=False))

results_accuracy.loc['1-level DTC', 'Bagging'] = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

In [ ]:
acc_mean, acc_std = eval_classifier(
    BaggingClassifier(DecisionTreeClassifier(), n_estimators=10, 
                      max_samples=1.0, max_features=0.8, 
                      bootstrap=True, bootstrap_features=False))

results_accuracy.loc['deep DTC', 'Bagging_with_mf'] = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

In [ ]:
acc_mean, acc_std = eval_classifier(
    BaggingClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=10, 
                      max_samples=1.0, max_features=0.8, 
                      bootstrap=True, bootstrap_features=False))

results_accuracy.loc['1-level DTC', 'Bagging_with_mf'] = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

In [ ]:
results_accuracy

**Задание**: Дополните таблицу: обучите также логистическую регрессию с беггингом и без него.

In [ ]:
acc_mean, acc_std = eval_classifier(LogisticRegression(max_iter=1000))

results_accuracy.loc['LR', 'W/o ensembling'] = acc_mean
print(f"LR W/o ensembling: {acc_mean:.2f}, +- {acc_std:.2f}")

In [ ]:
acc_mean, acc_std = eval_classifier(
    BaggingClassifier(LogisticRegression(max_iter=1000), n_estimators=10,
    max_samples=1.0, max_features=1.0,
    bootstrap=True, bootstrap_features=False))

results_accuracy.loc['LR', 'Bagging'] = acc_mean
print(f"LR with Bagging: {acc_mean:.2f}, +- {acc_std:.2f}")

In [ ]:
acc_mean, acc_std = eval_classifier(
    BaggingClassifier(LogisticRegression(max_iter=1000), n_estimators=10,
    max_samples=1.0, max_features=0.8,
    bootstrap=True, bootstrap_features=False))

results_accuracy.loc['LR', 'Bagging_with_mf'] = acc_mean
print(f"LR with Bagging_with_mf: {acc_mean:.2f}, +- {acc_std:.2f}")

In [ ]:
results_accuracy

## AdaBoost

AdaBoost обучает каждый следующий классификатор на объектах, на которых ошибаются предыдущие (объекты с ошибками получают больший вес, без ошибок — меньший).

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
acc_mean, acc_std = eval_classifier(
    AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=50, learning_rate=1.0))

results_accuracy.loc['deep DTC', 'AdaBoost'] = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

In [ ]:
acc_mean, acc_std = eval_classifier(
    AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=50, learning_rate=1.0))

results_accuracy.loc['1-level DTC', 'AdaBoost'] = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

In [ ]:
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

results_to_show = results_accuracy.copy()

results_to_show.style.apply(highlight_max, props='font-weight: bold;', axis=1).format('{:.3f}')

**Задание:** выясните, дадут ли улучшение бэггинг и бустинг над линейной регрессией. Объясните, почему так.

_Наверное, в задании имелась в виду логистическая регрессия, так как наш датасет создан как задача классификации. Поэтому добавим LR AdaBoost в табличку и оценим результаты._

In [ ]:
acc_mean, acc_std = eval_classifier(
    AdaBoostClassifier(
        estimator=LogisticRegression(max_iter=1000),
        n_estimators=50,
        learning_rate=1.0,
        random_state=42
    )
)

results_accuracy.loc['LR', 'AdaBoost'] = acc_mean
print(f"LR with AdaBoost: {acc_mean:.2f}, ± {acc_std:.2f}")

In [ ]:
results_to_show = results_accuracy.copy()

results_to_show.style.apply(highlight_max, props='font-weight: bold;', axis=1).format('{:.3f}')

_**Возможное объяснение результатов**:_ 

_Логистическая регрессия - линейная модель с низкой дисперсией. Бэггинг помогает уменьшать дисперсию, но у линейных моделей она и так минимальна. Усреднение множества линейных моделей дает такую же линейную модель._

_А что касается AdaBoost, он предназначен для слабых классификаторов (мелкие деревья), а не для линейных моделей, а логистическая регрессия и так дает оптимальное линейное решение. Поэтому попытки "бустить" линейную модель приводят к переобучению или ухудшению качества._

Вопрос: Почему AdaBoost хуже работает на глубоких деревьях?

_**Возможный ответ:**_

_Идея бустинга - это последовательное улучшение за счет исправления ошибок простых моделей. А глубокие деревья — уже сильные классификаторы, а потому на бустинге они вполне спокойно могут переобучаться._

# Градиентный бустинг

Рассмотрим композицию $a = \sum_{i} {\gamma_i b_i}$. Для начала выбирается какой-нибудь простой $\gamma_0, b_0$ (например, 0 и среднее). Формально каждая N-ая модель, начиная со второй, пытается приблизиить антиградиент функционала ошибки, взятый в точках ${z_i=a_{N-1}(x_i)}$:
$$s_i = -\dfrac{\partial L(y, z)}{\partial z}|_{z=a_{N-1}}$$
Подбор алгоритма при этом производится, приближая эту ошибку c точки зрения квадратичной функции потерь.
$$ b_i = arg \min_{b \in \mathcal{B}} {\sum (b_i(x) - s_i)^2} $$

Градиентный бустинг - довольно мощная метамодель, с огромным количеством параметров и хитростей. Мы сегодня остановимся только на основных. Для начала рассмотрим самый стандартный бустинг с использованием деревьев решений (CART). Параметры базовых моделей такие же, как и раньше, но настройка амого бустинга довольно сложна!

Важный вопрос при обучении модели - какую функцию ошибок выбрать? Какая задача возникает при обработке датасета с вином?

Для того, чтобы оценивать модель, полезны различные метрики - численные характеристики ее качества. При этом бустинги настолько галантны, что предоставляют нам возможность оценивать метрики прямо при обучении. Для этого необходимо задать тип метрики в конструкторе и eval_set при запуске fit().

На практике обычно используется один из трех вариантов бустинга - Xgboost, LightGBM или CatBoost.

### [XGBoost](https://www.kdd.org/kdd2016/papers/files/rfp0697-chenAemb.pdf).
Плюсы:
- Позволяет легко паралелить вычисления (например на спарке)
- Легко использовать с sklearn и numpy (но с потерей производительности)
- Поддерживается обработка разреженных данных
- Предсортированные блоки, кэши, шардирование

Минусы:
- Нет поддержки GPU

[документация](https://xgboost.readthedocs.io/en/latest/)

  
### [LightGBM](https://papers.nips.cc/paper/2017/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf)
Плюсы:
- Поддержка GPU
- Метод Фишера для работы с категориальными признаками
- Уменьшение размера обучающей выборки (GOSS)
- Объединение разреженных признаков (EFB)

Минусы:
- Итерфейс не совместим с sklearn/numpy

[документация](https://lightgbm.readthedocs.io/en/latest/Python-API.html)

### [CatBoost](https://papers.nips.cc/paper/2017/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf)
Плюсы:
- Поддержка GPU
- Легко использовать с sklearn и numpy
- Более продвинутая работа с категориальными фичами
- Наши слоны
  
Минусы:
- Бывает работает хуже (возможно слабее эвристики), но с категориальными фичами — хорошо

[документация](https://catboost.ai/docs/concepts/python-quickstart.html)


In [ ]:
from catboost import CatBoostClassifier

acc_mean, acc_std = eval_classifier(
    CatBoostClassifier(
        iterations=10,
        depth=1,
        learning_rate=1,
        loss_function='Logloss',
        verbose=True, 
        task_type='CPU'))

сat_boost = acc_mean

results_accuracy.loc['1-level DTC', 'CatBoost'] = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

In [ ]:
from catboost import CatBoostClassifier

acc_mean, acc_std = eval_classifier(
    CatBoostClassifier(
        iterations=10,
        learning_rate=1,
        loss_function='Logloss',
        verbose=True, 
        task_type='CPU'))

сat_boost = acc_mean

results_accuracy.loc['deep DTC', 'CatBoost'] = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

In [ ]:
from xgboost import XGBClassifier
acc_mean, acc_std = eval_classifier(XGBClassifier(objective='binary:logistic', random_state=42))

xg_boost = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

results_accuracy.loc['deep DTC', 'XGBoost'] = acc_mean

In [ ]:
acc_mean, acc_std = eval_classifier(XGBClassifier(objective='binary:logistic', max_depth=1, random_state=42))

xg_boost = acc_mean
print(f"{acc_mean:.2f}, {acc_std:.2f}")

results_accuracy.loc['1-level DTC', 'XGBoost'] = acc_mean

In [ ]:
results_to_show = results_accuracy.copy()
results_to_show.style.apply(highlight_max, props='font-weight: bold;', axis=1).format('{:.3f}')

In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(X, label=y)

param = {'num_leaves': 31, 
         'objective': 'multiclass', 
         'num_class': 2, 
         'metric': ['multi_logloss']}

num_round = 10
boost = lgb.train(param, train_data, num_boost_round=10)

lg_boost = (boost.predict(X).argmax(axis=-1) == y).mean()

results_accuracy.loc['deep DTC', 'LightGBM'] = lg_boost
print(f"{lg_boost:.2f}")

In [ ]:
results_to_show = results_accuracy.copy()

results_to_show.style.apply(highlight_max, props='font-weight: bold;', axis=1).format('{:.3f}')

## Пример
Рассмотрим реальный датасет, и на его примере попробуем поработать с бустингом.

In [ ]:
from sklearn import datasets
ds = datasets.load_diabetes()
X = ds.data
Y = ds.target
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.5, test_size=0.5)


In [ ]:
model = XGBRegressor(n_estimators=100, learning_rate=1, seed=SEED)
fit_params = {"eval_set":[(X_train, y_train),(X_test, y_test)], "verbose": False}
# Add verbose=False to avoid printing out updates with each cycle
model.fit(X_train, y_train,
            eval_set=[(X_train, y_train),(X_test, y_test)],
            verbose=False)

In [ ]:
results = model.evals_result()

In [ ]:
error_function = "rmse"
plt.figure(figsize=(10,7))
plt.plot(results["validation_0"][error_function], label="Training loss")
plt.plot(results["validation_1"][error_function], label="Validation loss")
plt.xlabel("Number of trees")
plt.ylabel("RMSE")
plt.legend()

Как мы видим, хотя лосс при обучении падал и падал, на валидации метрики перестали улучшаться довольно рано. Это довольно плохой знак. Однако говорит ли это о катастрофической ситуации? Проверим переобучение с помощью кросс-валидации.

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, X, Y, cv=10, scoring=["neg_root_mean_squared_error"],
                            return_train_score=True)
print("Train RMSE is", -cv_results['train_neg_root_mean_squared_error'].mean())
print("Test RMSE is", -cv_results['test_neg_root_mean_squared_error'].mean())


Кажется, у нас действительно серьезные проблемы. Попробуем уменьшить скорость обучения.


In [ ]:
# train and eval model with smaller lr
model = XGBRegressor(n_estimators=100, learning_rate=0.01, seed=SEED)
fit_params = {"eval_set":[(X_train, y_train),(X_test, y_test)], "verbose": False}
# Add verbose=False to avoid printing out updates with each cycle
model.fit(X_train, y_train,
            eval_set=[(X_train, y_train),(X_test, y_test)],
            verbose=False)

In [ ]:
# plot results
results = model.evals_result()
error_function = "rmse"
plt.figure(figsize=(10,7))
plt.plot(results["validation_0"][error_function], label="Training loss")
plt.plot(results["validation_1"][error_function], label="Validation loss")
plt.xlabel("Number of trees")
plt.ylabel("RMSE")
plt.legend()

Помогло ли это? Попробуем получить результаты лучше, поиграв с параметрами.

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, X, Y, cv=10, scoring=["neg_root_mean_squared_error"],
                            return_train_score=True)
print("Train RMSE is", -cv_results['train_neg_root_mean_squared_error'].mean())
print("Test RMSE is", -cv_results['test_neg_root_mean_squared_error'].mean())


In [ ]:
def train_and_evaluate(params, X_train, y_train, X_test, y_test, X, Y):
    label = params.pop("label")
    model = XGBRegressor(**params)

    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=False
    )

    results = model.evals_result()
    plt.figure(figsize=(10,6))
    plt.plot(results["validation_0"]["rmse"], label="Тренировочный")
    plt.plot(results["validation_1"]["rmse"], label="Валидационный")
    plt.title(f"Learning curve (params: {label})")
    plt.xlabel("Number of trees")
    plt.ylabel("RMSE")
    plt.grid(True)
    plt.legend()
    plt.show()

    model_cv = XGBRegressor(**params)
    cv_results = cross_validate(
        model_cv, X, Y,
        cv=5,
        scoring=["neg_root_mean_squared_error"],
        return_train_score=True
    )
    print("Train RMSE:", -cv_results['train_neg_root_mean_squared_error'].mean())
    print("Test RMSE:", -cv_results['test_neg_root_mean_squared_error'].mean())
    print("-" * 50)


In [ ]:
params1 = {
    "n_estimators": 1000,
    "learning_rate": 0.01,
    "max_depth": 6,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "gamma": 0,
    "reg_alpha": 0.0,
    "reg_lambda": 1.0,
    "random_state": SEED,
    "eval_metric": "rmse",
    "label": "params1"
}
train_and_evaluate(params1, X_train, y_train, X_test, y_test, X, Y)

In [ ]:
params2 = {
    "n_estimators": 1000,
    "learning_rate": 0.01,
    "max_depth": 4,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "gamma": 1,
    "reg_alpha": 0.1,
    "reg_lambda": 2.0,
    "random_state": SEED,
    "eval_metric": "rmse",
    "label": "params2"
}
train_and_evaluate(params2, X_train, y_train, X_test, y_test, X, Y)

In [ ]:
params3 = {
    "n_estimators": 1000,
    "learning_rate": 0.01,
    "max_depth": 5,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    "gamma": 0.5,
    "reg_alpha": 0.05,
    "reg_lambda": 1.5,
    "random_state": SEED,
    "eval_metric": "rmse",
    "label": "params3"
}
train_and_evaluate(params3, X_train, y_train, X_test, y_test, X, Y)

Так как параметров довольно много, может быть разумно автоматизировать их поиск. Для этого воспользуемся поиском по решетке.

In [ ]:
from sklearn.model_selection import GridSearchCV

model = XGBRegressor(
    objective="reg:squarederror",
    seed=SEED,
    verbosity=0
)
xgboost_params = {
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.9],
    "colsample_bytree": [0.7, 1.0],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [1, 2],
    "n_estimators": [300, 500]
}
fit_params = {
    "eval_set": [(X_train, y_train), (X_test, y_test)],
    "verbose": False
}

xgboost_best_grid = GridSearchCV(
    model,
    xgboost_params,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    return_train_score=True
)

xgboost_best_grid.fit(X_train, y_train, **fit_params)

In [ ]:
print("Best RMSE:", -xgboost_best_grid.best_score_)
print("Best parameters:")
for k, v in xgboost_best_grid.best_params_.items():
    print(f"  {k}: {v}")

Давайте проверим, какую точность мы получим с лучшими параметрами.

_Лучший результат показан в предыдущей ячейке._

**Задание:** Проведите обучение и с LightGBM/CatBoost. Какие лучшие точности у вас получилось получить?

In [ ]:
from lightgbm import LGBMRegressor

lgb_model = LGBMRegressor(random_state=SEED)

lgb_params = {
    "learning_rate": [0.01, 0.05],
    "max_depth": [3, 5],
    "n_estimators": [300, 500],
    "subsample": [0.8, 0.9],
    "colsample_bytree": [0.8, 1.0],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [1, 2]
}

lgb_grid = GridSearchCV(
    lgb_model, lgb_params,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1,
    return_train_score=True
)
lgb_grid.fit(X_train, y_train)

print("LightGBM:")
print("Best RMSE:", -lgb_grid.best_score_)
print("Best parameters:")
for k, v in lgb_grid.best_params_.items():
    print(f"  {k}: {v}")

**Задание:** Постройте графики предсказаний для первых двух PCA фичей для бустингов разной глубины/разного числа деревьев.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(
    X_pca, Y, train_size=0.8, random_state=SEED
)


In [ ]:
def plot_prediction_surface(model, X, y, title):
    h = 0.1
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, cmap="coolwarm", alpha=0.5)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap="coolwarm", edgecolors="k", s=50)
    plt.title(title)
    plt.xlabel("PCA 1")
    plt.ylabel("PCA 2")
    plt.colorbar(label="Предсказание")
    plt.show()

In [ ]:
for depth in [2, 4, 6]:
    for n_trees in [100, 300]:
        model = XGBRegressor(
            max_depth=depth,
            n_estimators=n_trees,
            learning_rate=0.05,
            random_state=SEED,
            verbosity=0
        )
        model.fit(X_train_pca, y_train_pca)
        title = f"XGBoost (depth={depth}, trees={n_trees})"
        plot_prediction_surface(model, X_pca, Y, title)


**Задание**: Постройте график зависимости точности от глубины

In [ ]:
depths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rmse_scores = []

for d in depths:
    model = XGBRegressor(
        max_depth=d,
        n_estimators=300,
        learning_rate=0.05,
        random_state=SEED,
        verbosity=0
    )
    scores = cross_val_score(model, X, Y,
                              cv=5,
                              scoring="neg_root_mean_squared_error")
    mean_rmse = -np.mean(scores)
    rmse_scores.append(mean_rmse)
    print(f"depth={d}, RMSE={mean_rmse:.4f}")


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(depths, rmse_scores, marker="o", linestyle="-")
plt.title("Зависимость RMSE от max_depth (XGBoost)")
plt.xlabel("max_depth")
plt.ylabel("RMSE")
plt.grid(True)